In [89]:
%%capture
!pip install pulp
!pip install matplotlib

In [90]:
from pulp import *
import matplotlib.pyplot as plt

### Dados de entrada

In [91]:
perfis = [
	{
		"nome": "Aposentados",
		"propostas_recebidas": 2000,
		"taxa_inadimplencia": 0.04,
		"taxa_juros": 0.015
	},
	{
		"nome": "Pensionistas",
		"propostas_recebidas": 1242,
		"taxa_inadimplencia": 0.11,
		"taxa_juros": 0.02
	},
	{
		"nome": "PJ",
		"propostas_recebidas": 3982,
		"taxa_inadimplencia": 0.06,
		"taxa_juros": 0.03
	},
	{
		"nome": "CLT",
		"propostas_recebidas": 218,
		"taxa_inadimplencia": 0.08,
		"taxa_juros": 0.025
	},
	{
		"nome": "Funcionários internos",
		"propostas_recebidas": 1802,
		"taxa_inadimplencia": 0.00,
		"taxa_juros": 0.01
	}
]

max_clientes = 5000

### Definição das variáveis

In [92]:
problema = LpProblem("Maximizar_Lucro", LpMaximize)

variaveis = LpVariable.dicts("Propostas", [perfil["nome"] for perfil in perfis], lowBound=0)

### Aplicação do cálculo sobre as características de cada perfil

In [93]:
problema += lpSum(
    (1 - perfil["taxa_inadimplencia"]) * perfil["taxa_juros"] * variaveis[perfil["nome"]]
    for perfil in perfis
)

problema += lpSum(variaveis) <= max_clientes

for perfil in perfis:
    problema += variaveis[perfil["nome"]] <= perfil["propostas_recebidas"]

In [ ]:
problema.solve()

### Visualização dos resultados

In [ ]:
for perfil in perfis:
    print(f"{perfil["nome"]} = {variaveis[perfil["nome"]].varValue}")

### Visualização gráfica

In [ ]:
nomes_perfis = [perfil["nome"] for perfil in perfis]
propostas_aceitas = [variaveis[perfil["nome"]].varValue for perfil in perfis]
propostas_recebidas = [perfil["propostas_recebidas"] for perfil in perfis]

plt.figure(figsize=(10, 6))
barWidth = 0.4
r1 = range(len(nomes_perfis))
r2 = [x + barWidth for x in r1]

plt.bar(r1, propostas_aceitas, color='#7aff70', width=barWidth, label='Propostas Aceitas')
plt.bar(r2, propostas_recebidas, color='#88f4fc', width=barWidth, label='Propostas Recebidas')

plt.xlabel('Perfis de Clientes')
plt.ylabel('Número de Propostas')
plt.xticks([r + barWidth/2 for r in range(len(nomes_perfis))], nomes_perfis)
plt.title('Número de Propostas Recebidas/Aceitas por Perfil')
plt.legend()
plt.show()